<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Задание 3

## Классификация текстов

В этом задании вам предстоит попробовать несколько методов, используемых в задаче классификации, а также понять насколько хорошо модель понимает смысл слов и какие слова в примере влияют на результат.

In [ ]:
#!pip list

In [1]:
from IPython.display import clear_output
!pip install -q pymorphy2
!pip install torchtext==0.8.1
clear_output()

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm, trange
import torch
import torchtext
print(torchtext.__version__)

#from torchtext.legacy import datasets 
from torchtext import datasets

from torchtext.data import Field, LabelField, BucketIterator
from torchtext.vocab import Vectors, GloVe

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from tqdm.autonotebook import tqdm
from sklearn.metrics import f1_score

0.8.1


В этом задании мы будем использовать библиотеку torchtext. Она довольна проста в использовании и поможет нам сконцентрироваться на задаче, а не на написании Dataloader-а.

In [5]:
TEXT = Field(sequential=True, lower=True, include_lengths=True)  # Поле текста
LABEL = LabelField(dtype=torch.float)  # Поле метки

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [3]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Датасет на котором мы будем проводить эксперементы это комментарии к фильмам из сайта IMDB.

In [7]:
train, test = torchtext.datasets.IMDB.splits(TEXT, LABEL)  # загрузим датасет
train, valid = train.split(random_state=random.seed(SEED))  # разобьем на части

/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [6]:
TEXT.build_vocab(train)
LABEL.build_vocab(train)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, valid_iter, test_iter = BucketIterator.splits(
    (train, valid, test), 
    batch_size = 64,
    sort_within_batch = True,
    device = device)

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [8]:
print(device)

cuda


## RNN

Для начала попробуем использовать рекурентные нейронные сети. На семинаре вы познакомились с GRU, вы можете также попробовать LSTM. Можно использовать для классификации как hidden_state, так и output последнего токена.

In [9]:
class RNNBaseline(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        self.bidirectional = bidirectional
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout)
        self.n_layers = n_layers
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
    
        self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers,  bidirectional=bidirectional)  # YOUR CODE GOES HERE
        
        self.fc = nn.Linear(hidden_dim*2, output_dim) # YOUR CODE GOES HERE
        
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        # cell arg for LSTM, remove for GRU
        #packed_output, hidden = self.rnn(packed_embedded)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)  

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        # YOUR CODE GOES HERE
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        hidden = self.dropout(hidden)
                
        #hidden = [batch size, hid dim * num directions] or [batch_size, hid dim * num directions]
            
        return self.fc(hidden)

Поиграйтесь с гиперпараметрами

In [10]:
vocab_size = len(TEXT.vocab)
emb_dim = 100
hidden_dim = 256
output_dim = 1
n_layers = 2
bidirectional = True
dropout = 0.1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
patience=3

In [11]:
model = RNNBaseline(
    vocab_size=vocab_size,
    embedding_dim=emb_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    n_layers=n_layers,
    bidirectional=bidirectional,
    dropout=dropout,
    pad_idx=PAD_IDX
)

In [12]:
model = model.to(device)

In [13]:
optimizer = torch.optim.Adam(model.parameters())
loss_func = nn.BCEWithLogitsLoss()

max_epochs = 6

Обучите сетку! Используйте любые вам удобные инструменты, Catalyst, PyTorch Lightning или свои велосипеды.

In [14]:
min_loss = np.inf
all_labels = []
all_predictions = []
cur_patience = 0
max_grad_norm=2

for epoch in trange(1, max_epochs + 1):
    train_losss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        #YOUR CODE GOES HERE
        optimizer.zero_grad()
        input_embeds = batch.text[0].to(device)
        text_length = batch.text[1].cpu()
        labels = torch.unsqueeze(batch.label, 1).to(device)
        prediction = model(input_embeds,text_length)
        train_loss = loss_func(prediction, labels)
        train_loss.backward()
        train_losss += train_loss.item()
        
        optimizer.step()
    train_losss /= len(train_iter)
    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(valid_iter), total=len(valid_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    with torch.no_grad():

      for it, batch in pbar:
    
        # YOUR CODE GOES HERE
        input_embeds = batch.text[0].to(device)
        text_length = batch.text[1].cpu()
        labels = torch.unsqueeze(batch.label, 1).to(device)
        label = batch.label.tolist()
        all_labels += label
        prediction = model(input_embeds, text_length)
        predictions = list(map(lambda x: 1 if x>0.5 else 0,  torch.flatten(prediction).tolist())) 
        all_predictions += predictions
        val_loss += loss_func(prediction, labels).item()
        f1_scores = f1_score(label, predictions,  average='weighted', zero_division=1)
    val_loss /= len(test_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {},  F1 score: {}'.format(epoch, train_losss, val_loss, f1_scores))
#model.load_state_dict(best_model)

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 1, Training Loss: 0.6408335415333727, Validation Loss: 0.17759507200906954,  F1 score: 0.6111111111111112


Epoch: 2, Training Loss: 0.5486175216680026, Validation Loss: 0.16877088171746724,  F1 score: 0.6857142857142856


Epoch: 3, Training Loss: 0.45993520254201264, Validation Loss: 0.14296499127164827,  F1 score: 0.9201680672268907


Epoch: 4, Training Loss: 0.33507391418853816, Validation Loss: 0.19596627255534882,  F1 score: 0.6857142857142856


Epoch: 5, Training Loss: 0.23753370447968045, Validation Loss: 0.14691422936861473,  F1 score: 0.8333333333333334


In [15]:
val_loss = 0.0
all_predictions = []
all_labels = []
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
pbar.set_description(f"Test")
with torch.no_grad():
    model.eval()
    for it, batch in pbar:
    
        # YOUR CODE GOES HERE
        input_embeds = batch.text[0].to(device)
        text_length = batch.text[1].cpu()
        labels = torch.unsqueeze(batch.label, 1).to(device)
        label = batch.label.tolist()
        all_labels += label
        prediction = model(input_embeds, text_length)
        predictions = list(map(lambda x: 1 if x>0 else 0,  torch.flatten(prediction).tolist())) 
        all_predictions += predictions
        val_loss += loss_func(prediction, labels).item()
        f1_scores = f1_score(label, predictions,  average='weighted', zero_division=1)
val_loss /= len(test_iter)
print('Epoch: {},  Test Loss: {}, f1-score: {}'.format(epoch, val_loss, f1_scores))

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 6,  Test Loss: 0.5481047851159749, f1-score: 0.7423273657289002


In [16]:
print(f"F1_score: {f1_score(all_labels, all_predictions, average='weighted')}")

F1_score: 0.7990676170243088


Посчитайте f1-score вашего классификатора на тестовом датасете.

**Ответ**: f1_score LSTM 0.7990676170243088



## CNN

![](https://www.researchgate.net/publication/333752473/figure/fig1/AS:769346934673412@1560438011375/Standard-CNN-on-text-classification.png)

Для классификации текстов также часто используют сверточные нейронные сети. Идея в том, что как правило сентимент содержат словосочетания из двух-трех слов, например "очень хороший фильм" или "невероятная скука". Проходясь сверткой по этим словам мы получим какой-то большой скор и выхватим его с помощью MaxPool. Далее идет обычная полносвязная сетка. Важный момент: свертки применяются не последовательно, а параллельно. Давайте попробуем!

In [8]:
TEXT = Field(sequential=True, lower=True, batch_first=True)  # batch_first тк мы используем conv  
LABEL = LabelField(batch_first=True, dtype=torch.float)

train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

TEXT.build_vocab(trn)
LABEL.build_vocab(trn)

device = "cuda" if torch.cuda.is_available() else "cpu"

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes fo

In [9]:
train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Вы можете использовать Conv2d с `in_channels=1, kernel_size=(kernel_sizes[0], emb_dim))` или Conv1d c `in_channels=emb_dim, kernel_size=kernel_size[0]`. Но хорошенько подумайте над shape в обоих случаях.

In [4]:
class CNN(nn.Module):
    def __init__(
        self,
        vocab_size,
        emb_dim,
        out_channels,
        kernel_sizes,
        dropout=0.5,
        stride=2,
    ):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, emb_dim)
      
        # YOUR CODE GOES HERE
        self.conv_0 = nn.Conv2d(in_channels = 1, 
                                out_channels = out_channels, 
                                kernel_size = (kernel_sizes[0], emb_dim),
                                stride=stride)
        
        self.conv_1 = nn.Conv2d(in_channels = 1, 
                                out_channels = out_channels, 
                                kernel_size = (kernel_sizes[1], emb_dim),
                                stride=stride)
        
        self.conv_2 = nn.Conv2d(in_channels = 1, 
                                out_channels = out_channels, 
                                kernel_size = (kernel_sizes[2], emb_dim),
                                stride=stride)
      
        self.fc = nn.Linear(len(kernel_sizes) * out_channels, 1)
        
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, text):
        #text = [batch size, sent len]
        embedded = self.embedding(text)
        #print('embedded1', embedded.size())
        
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1) # may be reshape here working
        
        #embedded = [batch size, 1, sent len, emb dim]
         
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))  # may be reshape here
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))  # may be reshape here
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
        
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]

        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
      
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))
      
        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [11]:
kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
out_channels=64
dropout = 0.2
dim = 300
stride = 2
model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=out_channels,
            kernel_sizes=kernel_sizes, dropout=dropout, stride=stride)

In [12]:
model.to(device)

CNN(
  (embedding): Embedding(202268, 300)
  (conv_0): Conv2d(1, 64, kernel_size=(3, 300), stride=(2, 2))
  (conv_1): Conv2d(1, 64, kernel_size=(4, 300), stride=(2, 2))
  (conv_2): Conv2d(1, 64, kernel_size=(5, 300), stride=(2, 2))
  (fc): Linear(in_features=192, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [17]:
optimizer = torch.optim.Adam(model.parameters())
loss_func = nn.BCEWithLogitsLoss()

In [15]:
max_epochs = 3

In [252]:
#torch.cuda.empty_cache()

Обучите!

In [19]:
min_loss = np.inf
all_labels = []
all_predictions = []
cur_patience = 0
patience = 3
for epoch in trange(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        #YOUR CODE GOES HERE
    
        optimizer.zero_grad()
        input_embeds = batch.text.to(device)
        labels = batch.label.unsqueeze(1).to(device)
        prediction = model(input_embeds) 
        
        loss = loss_func(prediction, labels)
        
        loss.backward()
        train_loss += loss.item()
        
        optimizer.step()
    train_loss /= len(train_iter)
    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(val_iter), total=len(val_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    with torch.no_grad():

      for it, batch in pbar:
    
        # YOUR CODE GOES HERE
        input_embeds = batch.text.to(device)
        
        labels = torch.unsqueeze(batch.label, 1).to(device)
        label = batch.label.tolist()
        all_labels += label
        prediction = model(input_embeds)
        predictions = list(map(lambda x: 1 if x>0.5 else 0,  torch.flatten(prediction).tolist())) 
        all_predictions += predictions
        val_loss += loss_func(prediction, labels).item()
        f1_scores = f1_score(label, predictions,  average='weighted', zero_division=1)
    val_loss /= len(val_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {},  F1 score: {}'.format(epoch, train_loss, val_loss, f1_scores))
model.load_state_dict(best_model)

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 1, Training Loss: 0.4176274833888033, Validation Loss: 0.4169085055589676,  F1 score: 0.8023923444976077


Epoch: 2, Training Loss: 0.2940528555943148, Validation Loss: 0.39571084280808766,  F1 score: 0.7994954646963139


Epoch: 3, Training Loss: 0.20044569943073023, Validation Loss: 0.35994617094596226,  F1 score: 0.8814354066985647



<All keys matched successfully>

In [240]:
len(all_predictions)

7500

In [20]:
val_loss = 0.0
all_predictions = []
all_labels = []
model.eval()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
pbar.set_description(f"Test")
with torch.no_grad():
    
    for it, batch in pbar:
    
        # YOUR CODE GOES HERE
        input_embeds = batch.text.to(device)
        labels = torch.unsqueeze(batch.label, 1).to(device)
        label = batch.label.tolist()
        all_labels += label
        prediction = model(input_embeds)
        predictions = list(map(lambda x: 1 if x>0.5 else 0,  torch.flatten(prediction).tolist())) #torch.squeeze(
        all_predictions += predictions
        val_loss += loss_func(prediction, labels).item()
        f1_scores = f1_score(label, predictions,  average='weighted', zero_division=1)
        #print('Epoch: {},  Test Loss: {}, f1-score: {}'.format(epoch, val_loss, f1_scores))
val_loss /= len(test_iter)
        
print('Epoch: {},  Test Loss: {}, f1-score: {}'.format(epoch, val_loss, f1_scores))

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 3,  Test Loss: 0.3624577029627197, f1-score: 0.9433962264150945


In [21]:
print(f"F1_score of CNN: {f1_score(all_labels, all_predictions, average='weighted')}")

F1_score of CNN: 0.8253676510354815


Посчитайте f1-score вашего классификатора.

**Ответ**: F1_score of CNN: 0.8253676510354815

## Интерпретируемость

Посмотрим, куда смотрит наша модель. Достаточно запустить код ниже.

In [21]:
!pip install -q captum

In [23]:
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)

In [24]:
def forward_with_softmax(inp):
    logits = model(inp)
    return torch.softmax(logits, 0)[0][1]

def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))


# accumulate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 7, label = 0):
    model.eval()
    text = [tok for tok in TEXT.tokenize(sentence)]
    if len(text) < min_len:
        text += ['pad'] * (min_len - len(text))
    indexed = [TEXT.vocab.stoi[t] for t in text]

    model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    # input_indices dim: [sequence_length]
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid(input_indices).item()
    pred_ind = round(pred)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=5000, return_convergence_delta=True)

    print('pred: ', LABEL.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            LABEL.vocab.itos[pred_ind],
                            LABEL.vocab.itos[label],
                            LABEL.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))

In [25]:
interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.81 ) , delta:  tensor([1.6100e-06], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.12 ) , delta:  tensor([3.3908e-05], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.96 ) , delta:  tensor([2.2175e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.02 ) , delta:  tensor([3.5909e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.10 ) , delta:  tensor([2.8328e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.16 ) , delta:  tensor([0.0002], device='cuda:0', dtype=torch.float64)


Попробуйте добавить свои примеры!

In [26]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.81),pos,1.72,It was a fantastic performance ! pad
pos,neg (0.12),pos,-1.36,Best film ever pad pad pad pad
pos,pos (0.96),pos,0.90,Such a great show! pad pad pad
neg,neg (0.02),pos,-0.56,It was a horrible movie pad pad
neg,neg (0.10),pos,-0.18,I've never watched something as bad pad
neg,neg (0.16),pos,-0.09,It is a disgusting movie! pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.81),pos,1.72,It was a fantastic performance ! pad
pos,neg (0.12),pos,-1.36,Best film ever pad pad pad pad
pos,pos (0.96),pos,0.90,Such a great show! pad pad pad
neg,neg (0.02),pos,-0.56,It was a horrible movie pad pad
neg,neg (0.10),pos,-0.18,I've never watched something as bad pad
neg,neg (0.16),pos,-0.09,It is a disgusting movie! pad pad


## Эмбэдинги слов

Вы ведь не забыли, как мы можем применить знания о word2vec и GloVe. Давайте попробуем!

In [5]:
TEXT = Field(sequential=True, lower=True, batch_first=True)  # batch_first тк мы используем conv  
LABEL = LabelField(batch_first=True, dtype=torch.float)

train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))


device = "cuda" if torch.cuda.is_available() else "cpu"

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes fo

In [6]:
train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [7]:
TEXT.build_vocab(trn, vectors='glove.6B.300d')    #glove.6B.100d # YOUR CODE GOES HERE
# подсказка: один из импортов пока не использовался, быть может он нужен в строке выше :)
LABEL.build_vocab(trn)


In [13]:
kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
dropout = 0.5
dim = 300
stride = 2
model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=64,
            kernel_sizes=kernel_sizes, dropout=dropout, stride=stride)
optimizer = torch.optim.Adam(model.parameters())
loss_func = nn.BCEWithLogitsLoss()


CNN(
  (embedding): Embedding(202268, 300)
  (conv_0): Conv2d(1, 64, kernel_size=(3, 300), stride=(2, 2))
  (conv_1): Conv2d(1, 64, kernel_size=(4, 300), stride=(2, 2))
  (conv_2): Conv2d(1, 64, kernel_size=(5, 300), stride=(2, 2))
  (fc): Linear(in_features=192, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [14]:
word_embeddings = TEXT.vocab.vectors

model.embedding.weight = torch.nn.Parameter(word_embeddings) # инициализируйте эмбэдинги
prev_shape = model.embedding.weight.shape
assert prev_shape == model.embedding.weight.shape


In [16]:
max_epochs = 10
model.to(device)

CNN(
  (embedding): Embedding(202268, 300)
  (conv_0): Conv2d(1, 64, kernel_size=(3, 300), stride=(2, 2))
  (conv_1): Conv2d(1, 64, kernel_size=(4, 300), stride=(2, 2))
  (conv_2): Conv2d(1, 64, kernel_size=(5, 300), stride=(2, 2))
  (fc): Linear(in_features=192, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

Вы знаете, что делать.

In [17]:
min_loss = np.inf
all_labels = []
all_predictions =[]
cur_patience = 0
patience = 3
for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        #YOUR CODE GOES HERE
        optimizer.zero_grad()
        input_embeds = batch.text.to(device)
        labels = batch.label.unsqueeze(1).to(device)
        prediction = model(input_embeds)
        loss = loss_func(prediction, labels)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    train_loss /= len(train_iter)
    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(val_iter), total=len(val_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    with torch.no_grad():
      
      for it, batch in pbar:
        # YOUR CODE GOES HERE

        input_embeds = batch.text.to(device)
        
        labels = batch.label.unsqueeze(1).to(device) 
        label = batch.label.tolist()
        all_labels += label
        prediction = model(input_embeds)
        predictions = list(map(lambda x: 1 if x>0.5 else 0,  torch.flatten(prediction).tolist())) 
        all_predictions += predictions
        val_loss += loss_func(prediction, labels).item()
        f1_scores = f1_score(label, predictions,  average='weighted', zero_division=1)
    val_loss /= len(val_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}, f1_score: {}'.format(epoch, train_loss, val_loss, f1_scores))
model.load_state_dict(best_model)

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 1, Training Loss: 0.5423646534446382, Validation Loss: 0.40481834709644315, f1_score: 0.8553884711779447


Epoch: 2, Training Loss: 0.3909091523093899, Validation Loss: 0.3617334455251694, f1_score: 0.8680563154247366


Epoch: 3, Training Loss: 0.34381377696990967, Validation Loss: 0.34914378126462303, f1_score: 0.8680563154247366


Epoch: 4, Training Loss: 0.30232666672146236, Validation Loss: 0.3430729568004608, f1_score: 0.8948827369880002


Epoch: 5, Training Loss: 0.2765498015567334, Validation Loss: 0.3421104192733765, f1_score: 0.8814354066985647


Epoch: 6, Training Loss: 0.24227805348643422, Validation Loss: 0.3380736142396927, f1_score: 0.8680563154247366


Epoch: 7, Training Loss: 0.21694237439736833, Validation Loss: 0.3412270834048589, f1_score: 0.8948827369880002


Epoch: 8, Training Loss: 0.19593798091811856, Validation Loss: 0.3442024846871694, f1_score: 0.8686038011695906


<All keys matched successfully>

In [18]:
test_loss = 0.0
all_predictions = []
all_labels = []
model.eval()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
pbar.set_description(f"Test")
with torch.no_grad():
    
    for it, batch in pbar:
    
        # YOUR CODE GOES HERE
        input_embeds = batch.text.to(device)
        labels = torch.unsqueeze(batch.label, 1).to(device)
        label = batch.label.tolist()
        all_labels += label
        prediction = model(input_embeds)
        predictions = list(map(lambda x: 1 if x>0.5 else 0,  torch.flatten(prediction).tolist())) #torch.squeeze(
        all_predictions += predictions
        test_loss += loss_func(prediction, labels).item()
        f1_scores = f1_score(label, predictions,  average='weighted', zero_division=1)
    test_loss /= len(test_iter)
      
    print('Epoch: {},  Test Loss: {}, f1-score: {}'.format(epoch, test_loss, f1_scores))

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 9,  Test Loss: 0.337499308951047, f1-score: 0.975609756097561


In [19]:
print(f"F1_score of CNN with glove: {f1_score(all_labels, all_predictions, average='weighted')}")

F1_score of CNN with glove: 0.8474551764025448


Посчитайте f1-score вашего классификатора.

**Ответ** F1_score of CNN with glove: 0.8474551764025448



Проверим насколько все хорошо!

In [25]:
PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)
vis_data_records_ig = []

interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.96 ) , delta:  tensor([2.3853e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.15 ) , delta:  tensor([7.8928e-06], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.64 ) , delta:  tensor([1.4183e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.01 ) , delta:  tensor([0.0002], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.24 ) , delta:  tensor([1.3745e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.01 ) , delta:  tensor([4.1279e-07], device='cuda:0', dtype=torch.float64)


In [26]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.96),pos,1.74,It was a fantastic performance ! pad
pos,neg (0.15),pos,0.17,Best film ever pad pad pad pad
pos,pos (0.64),pos,1.34,Such a great show! pad pad pad
neg,neg (0.01),pos,-0.76,It was a horrible movie pad pad
neg,neg (0.24),pos,0.63,I've never watched something as bad pad
neg,neg (0.01),pos,-0.76,It is a disgusting movie! pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.96),pos,1.74,It was a fantastic performance ! pad
pos,neg (0.15),pos,0.17,Best film ever pad pad pad pad
pos,pos (0.64),pos,1.34,Such a great show! pad pad pad
neg,neg (0.01),pos,-0.76,It was a horrible movie pad pad
neg,neg (0.24),pos,0.63,I've never watched something as bad pad
neg,neg (0.01),pos,-0.76,It is a disgusting movie! pad pad


# Вывод


Итак, лучшей моделью оказалась CNN + glove с f1_score,  равным 0.84.
 LSTM дала результат 0.79, CNN с 3 conv2d - 0.82, CNN + glove - 0.84.